# My mushroom data classifier

## Technique
I will use a random forest model, and with that take various peices of user data to give my best guess as to if a mushroom is edible or not.

## Data
I will use a kaggle dataset for this

[Mushroom classification](https://www.kaggle.com/uciml/mushroom-classification) : UCI Machine Learning : CC0: Public Domain

## Plan
I will train this model, and then hopefully export it to another process where I will use it in a program to take user data, and then classify it.

In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

## Data preprocessing

In [2]:
# Importing the dataset

df = pd.read_csv("mushrooms.csv")
df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [3]:
# Using one of my FAVORITE sklearn tool for letter data
def le(x):
    LE = LabelEncoder()
    LE.fit(x)
    return LE.transform(x)

for c in df.columns:
    df[str(c)] = le(df[str(c)])

df.head()
# Now we have numerical data.

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1


## Model Time
Im going to use a random forest model, which has quickly become one of my favorite bagging type ML models. I have used it on a large project for the school of optometry recently, and just its ease of use alone has made me very prone to using it for simple binary classification problems!!

In [32]:
x = df.drop(['class'], axis=1)  #Dropping the labels!
y = df['class']                 #dropping the features!

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=.25, random_state=25)   #Bigger dataset allows for more training margin

In [33]:
print(X_train.shape)
print(X_test.shape)

(6093, 22)
(2031, 22)


In [41]:
rf = RandomForestClassifier(max_depth=10, random_state=25) #6 and 25 are arbitrary here 
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, random_state=25)

In [42]:
y_pred = rf.predict(X_test)
print(f'Accuracy: {metrics.accuracy_score(y_test, y_pred)}')
print(f'F1: {metrics.f1_score(y_test, y_pred)}')

Accuracy: 1.0
F1: 1.0


## The result of 1.0
This is a concerning result, and could possibly be a case of over fitting, I will prune down until I get a result less than 1.


In [66]:
rf = RandomForestClassifier(max_depth=4, random_state=25) #4 is the lowest number that gives higher than .98, and 25 is arbitrary here 
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=4, random_state=25)

In [67]:
y_pred = rf.predict(X_test)
print(f'Accuracy: {metrics.accuracy_score(y_test, y_pred)}')
print(f'F1: {metrics.f1_score(y_test, y_pred)}')

Accuracy: 0.9901526341703595
F1: 0.9897435897435898


# Result
From this result of .989, we can see that our model is well suited for this application. 
After some revision, I settled on a useful model. The high accuracy and f1 score might indicate overfitting, however, I did some pruning to attempt to mitigate what overfitting there was.
(I started with clustering, and then desicion tree, and then finally remembered RFC! I cannot belive I didn't use this sooner!)